In [1]:
# Escolha o arquivo sobre o qual deseja-se calcular as previsões
caminhoPrevisao = './planilhasTeste/dados.csv'

# Escolha o nome do arquivo com os resultados das previsões
caminhoResultado = './resultadoPrevisoes/resultadoDados.csv'

In [2]:
# Realizando os imports
import pandas as pd
import math
import lightgbm
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score


# ESCOLHA A BASE DE DADOS PARA PREVER O TEMPO DE ENTREGA MUDANDO O DIRETÓRIO DO ARQUIVO ABAIXO
#                              vvvvvvvv

# Importando a base que deseja-se calcular as previsões
dfPrevisao = pd.read_csv(caminhoPrevisao)

# Importando a base onde o modelo será treinado
df = pd.read_csv('./baseDeDados/dataBaseTratada.csv')

# Importando a base utilizada para completar as informações ausentes de cidade e estado
dfCEP = pd.read_excel('./baseDeDados/listaCEP.xlsx')


In [3]:
# Importando tabelas adjacentes que contém informações relevantes sobre os pedidos 
customers = pd.read_csv('./planilhasTeste/customers.csv') 
order_items = pd.read_csv('./planilhasTeste/order_itens.csv') 
order_payments = pd.read_csv('./planilhasTeste/order_payments.csv') 
products = pd.read_csv('./planilhasTeste/products.csv')
sellers = pd.read_csv('./planilhasTeste/sellers.csv') 

In [4]:
# Eliminando linhas duplicadas para não ter problema na hora de dar merge 
customers.drop_duplicates(subset='customer_id', inplace=True)
order_items.drop_duplicates(subset='order_id', inplace=True)
order_payments.drop_duplicates(subset='order_id', inplace=True)

In [5]:
# Função para verificar e realizar merge apenas se houver exatamente uma coluna em comum
def merge_if_one_column_in_common(df1, df2, on):
    common_columns = set(df1.columns).intersection(set(df2.columns))
    if len(common_columns) == 1 and list(common_columns)[0] in on:
        return pd.merge(df1, df2, on=on, how='left')
    return df1

# Dando o merge
dfPrevisao = merge_if_one_column_in_common(dfPrevisao, customers, ['customer_id'])
dfPrevisao = merge_if_one_column_in_common(dfPrevisao, order_items, ['order_id'])
dfPrevisao = merge_if_one_column_in_common(dfPrevisao, order_payments, ['order_id'])
dfPrevisao = merge_if_one_column_in_common(dfPrevisao, products, ['product_id'])
dfPrevisao = merge_if_one_column_in_common(dfPrevisao, sellers, ['seller_id'])

pd.set_option('display.max_columns', None)

In [6]:
# Consertando os nomes das colunas caso necessário

# Lista de nomes de colunas corretos
nomes_corretos = ['order_id', 'customer_id', 'order_purchase_timestamp', 'order_approved_at', 
                    'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state', 
                    'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 
                    'freight_value', 'payment_sequential', 'payment_type', 'payment_installments', 
                    'payment_value', 'product_category_name', 'product_name_lenght', 'product_description_lenght', 
                    'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 
                    'product_width_cm', 'seller_zip_code_prefix', 'seller_city', 'seller_state']

# Função para encontrar a coluna correspondente ignorando a última palavra
def find_matching_column(incorrect_col, correct_columns):
    # Primeira tentativa: encontrar uma coluna que comece exatamente com o nome incorreto
    for col in correct_columns:
        if col.startswith(incorrect_col):
            return col
    
    # Segunda tentativa: encontrar uma coluna que tenha a mesma base, ignorando o último segmento
    incorrect_base = "_".join(incorrect_col.split('_')[:-1])
    for col in correct_columns:
        if col.startswith(incorrect_base):
            return col
    
    # Terceira tentativa: encontrar uma coluna que seja uma correspondência aproximada (base + última palavra)
    for col in correct_columns:
        if incorrect_col[:-1] == col:
            return col
    
    return None

# Criar um novo DataFrame vazio com as colunas corretas
dfTeste = pd.DataFrame(columns=nomes_corretos)

# Usar loc para copiar os dados relevantes do DataFrame original para o novo DataFrame usando os nomes corretos
for col in dfPrevisao.columns:
    if col in dfTeste.columns:
        dfTeste[col] = dfPrevisao[col]
    else:
        matching_col = find_matching_column(col, dfTeste.columns)
        if matching_col:
            dfTeste[matching_col] = dfPrevisao[col]

In [7]:
for i in range(0, len(dfTeste)):
    if str(dfTeste['customer_state'][i]) == "AC":
        dfTeste['customer_state'][i] = "acre"
    elif str(dfTeste['customer_state'][i]) == "AL":
        dfTeste['customer_state'][i] = "alagoas"
    elif str(dfTeste['customer_state'][i]) == "AP":
        dfTeste['customer_state'][i] = "amapa"
    elif str(dfTeste['customer_state'][i]) == "AM":
        dfTeste['customer_state'][i] = "amazonas"
    elif str(dfTeste['customer_state'][i]) == "BA":
        dfTeste['customer_state'][i] = "bahia"
    elif str(dfTeste['customer_state'][i]) == "CE":
        dfTeste['customer_state'][i] = "ceara"
    elif str(dfTeste['customer_state'][i]) == "ES":
        dfTeste['customer_state'][i] = "espirito santo"
    elif str(dfTeste['customer_state'][i]) == "GO":
        dfTeste['customer_state'][i] = "goias"
    elif str(dfTeste['customer_state'][i]) == "MA":
        dfTeste['customer_state'][i] = "maranhao"
    elif str(dfTeste['customer_state'][i]) == "MT":
        dfTeste['customer_state'][i] = "mato grosso"
    elif str(dfTeste['customer_state'][i]) == "MS":
        dfTeste['customer_state'][i] = "mato grosso do sul"
    elif str(dfTeste['customer_state'][i]) == "MG":
        dfTeste['customer_state'][i] = "minas gerais"
    elif str(dfTeste['customer_state'][i]) == "PA":
        dfTeste['customer_state'][i] = "para"
    elif str(dfTeste['customer_state'][i]) == "PB":
        dfTeste['customer_state'][i] = "paraiba"
    elif str(dfTeste['customer_state'][i]) == "PR":
        dfTeste['customer_state'][i] = "parana"
    elif str(dfTeste['customer_state'][i]) == "PE":
        dfTeste['customer_state'][i] = "pernambuco"
    elif str(dfTeste['customer_state'][i]) == "PI":
        dfTeste['customer_state'][i] = "piaui"
    elif str(dfTeste['customer_state'][i]) == "RJ":
        dfTeste['customer_state'][i] = "rio de janeiro"
    elif str(dfTeste['customer_state'][i]) == "RN":
        dfTeste['customer_state'][i] = "rio grande do norte"
    elif str(dfTeste['customer_state'][i]) == "RS":
        dfTeste['customer_state'][i] = "rio grande do sul"
    elif str(dfTeste['customer_state'][i]) == "RO":
        dfTeste['customer_state'][i] = "rondonia"
    elif str(dfTeste['customer_state'][i]) == "RR":
        dfTeste['customer_state'][i] = "roraima"
    elif str(dfTeste['customer_state'][i]) == "SC":
        dfTeste['customer_state'][i] = "santa catarina"
    elif str(dfTeste['customer_state'][i]) == "SP":
        dfTeste['customer_state'][i] = "sao paulo"
    elif str(dfTeste['customer_state'][i]) == "SE":
        dfTeste['customer_state'][i] = "sergipe"
    elif str(dfTeste['customer_state'][i]) == "TO":
        dfTeste['customer_state'][i] = "tocantins"
    elif str(dfTeste['customer_state'][i]) == "DF":
        dfTeste['customer_state'][i] = "distrito federal"

C:\Users\leopa\AppData\Local\Temp\ipykernel_19532\1195370365.py:37: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dfTeste['customer_state'][i] = "rio de janeiro"
C:\Users\leopa\AppData\Local\Temp\ipykernel_19532\1195370365.py:37: SettingWith

In [8]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Raio da Terra em km
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R * c

# Função para garantir que os valores sejam numéricos
def to_numeric(value, default=0.0):
    try:
        return float(value)
    except (ValueError, TypeError):
        return default

# Aplicar a função às colunas de latitude e longitude no dfCEP
dfCEP['Latitude'] = dfCEP['Latitude'].apply(to_numeric)
dfCEP['Longitude'] = dfCEP['Longitude'].apply(to_numeric)


# Supondo que dfTeste e dfCEP sejam os DataFrames fornecidos
listaLatitudeComprador = []
listaLongitudeComprador = []
listaLatitudeVendedor = []
listaLongitudeVendedor = []
listaDistancia = []

for i in range(len(dfTeste)):
    latitude_comprador = longitude_comprador = latitude_vendedor = longitude_vendedor = None

    for j in range(len(dfCEP)):
        if (dfTeste['customer_zip_code_prefix'][i] >= dfCEP['CEP Inicial'][j]) and (dfTeste['customer_zip_code_prefix'][i] <= dfCEP['CEP Final'][j]):
            try:
                latitude_comprador = float(dfCEP['Latitude'][j])
                longitude_comprador = float(dfCEP['Longitude'][j])
            except ValueError:
                latitude_comprador = 0
                longitude_comprador = 0
        if (dfTeste['seller_zip_code_prefix'][i] >= dfCEP['CEP Inicial'][j]) and (dfTeste['seller_zip_code_prefix'][i] <= dfCEP['CEP Final'][j]):
            try:
                latitude_vendedor = float(dfCEP['Latitude'][j])
                longitude_vendedor = float(dfCEP['Longitude'][j])
            except ValueError:
                latitude_vendedor = 0
                longitude_vendedor = 0

    listaLatitudeComprador.append(latitude_comprador if latitude_comprador is not None else 0)
    listaLongitudeComprador.append(longitude_comprador if longitude_comprador is not None else 0)
    listaLatitudeVendedor.append(latitude_vendedor if latitude_vendedor is not None else 0)
    listaLongitudeVendedor.append(longitude_vendedor if longitude_vendedor is not None else 0)

    if latitude_comprador is not None and longitude_comprador is not None and latitude_vendedor is not None and longitude_vendedor is not None:
        distancia = haversine(latitude_comprador, longitude_comprador, latitude_vendedor, longitude_vendedor)
    else:
        distancia = 0

    listaDistancia.append(distancia)

# Adicionando colunas ao DataFrame
dfTeste['Latitude_Comprador'] = listaLatitudeComprador
dfTeste['Longitude_Comprador'] = listaLongitudeComprador
dfTeste['Latitude_Vendedor'] = listaLatitudeVendedor
dfTeste['Longitude_Vendedor'] = listaLongitudeVendedor
dfTeste['Distancia'] = listaDistancia

In [9]:
for i in range(0, len(dfTeste)):
    dfTeste['customer_city'][i] = str(dfTeste['customer_city'][i]).lower()
    dfTeste['customer_state'][i] = str(dfTeste['customer_state'][i]).lower()

C:\Users\leopa\AppData\Local\Temp\ipykernel_19532\3598225989.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dfTeste['customer_city'][i] = str(dfTeste['customer_city'][i]).lower()
C:\Users\leopa\AppData\Local\Temp\ipykernel_19532\3598225

In [10]:
def ajustar_formato(data_str):
    try:
        partes = data_str.split(' ')
        data = partes[0].split('/')
        hora = partes[1]
        # Verificar se a data está no formato esperado
        if len(data) == 3 and len(partes) == 2:
            data_corrigida = data[2] + '-' + data[1] + '-' + data[0] + ' ' + hora
            return data_corrigida
        else:
            return data_str  # Retornar a string original se não for possível ajustar
    except:
        return data_str  # Em caso de erro, retornar a string original

# Aplicar a função de ajuste às colunas de datas
for coluna in ['shipping_limit_date', 'order_purchase_timestamp', 'order_approved_at']:
    dfTeste[coluna] = dfTeste[coluna].apply(ajustar_formato)

# Convertendo as colunas de data para datetime
dfTeste['shipping_limit_date'] = pd.to_datetime(dfTeste['shipping_limit_date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
dfTeste['order_purchase_timestamp'] = pd.to_datetime(dfTeste['order_purchase_timestamp'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
dfTeste['order_approved_at'] = pd.to_datetime(dfTeste['order_approved_at'], format='%Y-%m-%d %H:%M:%S', errors='coerce')


In [11]:
dfTeste['customer_zip_code_prefix'] = pd.to_numeric(dfTeste['customer_zip_code_prefix'], errors = 'coerce')
dfTeste['order_item_id'] = pd.to_numeric(dfTeste['order_item_id'], errors = 'coerce')
dfTeste['price'] = pd.to_numeric(dfTeste['price'], errors = 'coerce')
dfTeste['freight_value'] = pd.to_numeric(dfTeste['freight_value'], errors = 'coerce')
dfTeste['payment_sequential'] = pd.to_numeric(dfTeste['payment_sequential'], errors = 'coerce')
dfTeste['payment_installments'] = pd.to_numeric(dfTeste['payment_installments'], errors = 'coerce')
dfTeste['payment_value'] = pd.to_numeric(dfTeste['payment_value'], errors = 'coerce')
dfTeste['product_name_lenght'] = pd.to_numeric(dfTeste['product_name_lenght'], errors = 'coerce')
dfTeste['product_description_lenght'] = pd.to_numeric(dfTeste['product_description_lenght'], errors = 'coerce')
dfTeste['product_photos_qty'] = pd.to_numeric(dfTeste['product_photos_qty'], errors = 'coerce')
dfTeste['product_weight_g'] = pd.to_numeric(dfTeste['product_weight_g'], errors = 'coerce')
dfTeste['product_length_cm'] = pd.to_numeric(dfTeste['product_length_cm'], errors = 'coerce')
dfTeste['product_height_cm'] = pd.to_numeric(dfTeste['product_height_cm'], errors = 'coerce')
dfTeste['product_width_cm'] = pd.to_numeric(dfTeste['product_width_cm'], errors = 'coerce')
dfTeste['seller_zip_code_prefix'] = pd.to_numeric(dfTeste['seller_zip_code_prefix'], errors = 'coerce')

In [12]:
for coluna in dfTeste.columns:
    if (str(dfTeste[coluna].dtypes)) == 'float64' or (str(dfTeste[coluna].dtypes)) == 'int64':
        dfTeste[coluna].fillna(dfTeste[coluna].mean(), inplace = True)
    elif dfTeste[coluna].dtype == 'datetime64[ns]':
        dfTeste[coluna].fillna(pd.NaT, inplace=True)
    else:
        dfTeste[coluna].fillna("Ausente", inplace = True)

C:\Users\leopa\AppData\Local\Temp\ipykernel_19532\964266307.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfTeste[coluna].fillna("Ausente", inplace = True)
C:\Users\leopa\AppData\Local\Temp\ipykernel_19532\964266307.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

In [13]:
dfTeste['product_category_name'] = dfTeste['product_category_name'].astype('category')
dfTeste['payment_type'] = dfTeste['payment_type'].astype('category')
dfTeste['customer_city'] = dfTeste['customer_city'].astype('category')
dfTeste['customer_state'] = dfTeste['customer_state'].astype('category')
dfTeste['seller_city'] = dfTeste['seller_city'].astype('category')
dfTeste['seller_state'] = dfTeste['seller_state'].astype('category')

In [14]:
estados_vendedor_selecionados = ['SP', 'MG', 'RJ']

# Verificar quais estados estão presentes no DataFrame
estados_presentes = dfTeste['seller_state'].str.upper().unique()

# Filtrar apenas os estados presentes na lista de estados selecionados
estados_para_transformar = [estado for estado in estados_vendedor_selecionados if estado in estados_presentes]

# Aplicar one-hot encoding apenas às categorias presentes
if estados_para_transformar:
    dfTeste['seller_state'] = dfTeste['seller_state'].str.upper()
    df_one_hot = pd.get_dummies(dfTeste['seller_state'], prefix='estado_vendedor')[['estado_vendedor_' + cat for cat in estados_para_transformar]]
    df_one_hot = df_one_hot.astype(int)

    # Juntar as colunas one-hot com o DataFrame original
    dfTeste = pd.concat([dfTeste, df_one_hot], axis=1)

In [15]:
estados_comprador_selecionados = ['sao paulo', 'rio de janeiro', 'minas gerais']

# Verificar quais estados estão presentes no DataFrame
estados_presentes = dfTeste['customer_state'].str.lower().unique()

# Filtrar apenas os estados presentes na lista de estados selecionados
estados_para_transformar = [estado for estado in estados_comprador_selecionados if estado in estados_presentes]

# Aplicar one-hot encoding apenas às categorias presentes
if estados_para_transformar:
    df_one_hot = pd.get_dummies(dfTeste['customer_state'], prefix='estado_comprador')[['estado_comprador_' + cat for cat in estados_para_transformar]]
    df_one_hot = df_one_hot.astype(int)

# Juntar as colunas one-hot com o DataFrame original
dfTeste = pd.concat([dfTeste, df_one_hot], axis=1)

In [16]:
categorical_features = ['customer_state', 'seller_state']

X_df = df[['price', 'Latitude_Comprador', 'Longitude_Comprador', 'Latitude_Vendedor', 'Longitude_Vendedor','dia_da_semana_Thursday','dia_da_semana_Tuesday', 'dia_da_semana_Wednesday', 'dia_da_semana_Friday', 'dia_da_semana_Monday',	'dia_da_semana_Saturday', 'dia_da_semana_Sunday','freight_value', 'Distancia', 'product_weight_g', 'product_width_cm', 'product_height_cm', 'product_length_cm'] + categorical_features]
Y_df = df['delivery_time']

for col in categorical_features:
    X_df[col] = X_df[col].astype('category')

X_df_train = X_df
Y_df_train = Y_df

regressor_lgbm_dart = LGBMRegressor(boosting_type='dart', learning_rate=0.1, max_bin=255, reg_alpha=0.1)


cv_score = cross_val_score(regressor_lgbm_dart, X_df_train, Y_df_train, scoring='neg_root_mean_squared_error').mean()


regressor_campeao = regressor_lgbm_dart
regressor_campeao.fit(X_df_train, Y_df_train, eval_metric='rmse', categorical_feature=categorical_features)


coluna = dfTeste[['order_id']]


expected_columns = X_df.columns
for col in expected_columns:
    if col not in dfTeste.columns:
        dfTeste[col] = 0


X_final = dfTeste[expected_columns]


for col in categorical_features:
    X_final[col] = X_final[col].astype('category')


X_final['previsoes'] = regressor_campeao.predict(X_final)


dfResultado = pd.concat([coluna, X_final[['previsoes']]], axis=1)


dfResultado.to_csv(caminhoResultado, index=False)

dfResultado


C:\Users\leopa\AppData\Local\Temp\ipykernel_19532\571421577.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_df[col] = X_df[col].astype('category')
C:\Users\leopa\AppData\Local\Temp\ipykernel_19532\571421577.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_df[col] = X_df[col].astype('category')


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016549 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2273
[LightGBM] [Info] Number of data points in the train set: 58711, number of used features: 20
[LightGBM] [Info] Start training from score 13.191794
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016862 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2278
[LightGBM] [Info] Number of data points in the train set: 58711, number of used features: 20
[LightGBM] [Info] Start training from score 13.239586


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012151 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2282
[LightGBM] [Info] Number of data points in the train set: 58711, number of used features: 20
[LightGBM] [Info] Start training from score 13.214733
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002981 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2277
[LightGBM] [Info] Number of data points in the train set: 58711, number of used features: 20
[LightGBM] [Info] Start training from score 13.210013
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006467 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total 

C:\Users\leopa\AppData\Local\Temp\ipykernel_19532\571421577.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_final[col] = X_final[col].astype('category')
C:\Users\leopa\AppData\Local\Temp\ipykernel_19532\571421577.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_final[col] = X_final[col].astype('category')
C:\Users\leopa\AppData\Local\Temp\ipykernel_19532\571421577.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

order_id  previsoes
0  12018f77f2f0320c557190d7a144bdd3  13.148420
1  32018f77f2f0320c557190d7a144bdd3   7.728634
2  54018f77f2f0320c557190d7a144bdd3  14.254438
3  42018f77f2f0320c557190d7a144bdd3  10.091683
4  54018f77f2f0320c557190d7a144bdd3  16.508797
5  67018f77f2f0320c557190d7a144bdd3  10.292001
6  18018f77f2f0320c557190d7a144bdd3  13.276560
7  73018f77f2f0320c557190d7a144bdd3  10.040309